In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config
from safetensors.torch import load_file
import torch
import os
import datasets
from tqdm.auto import tqdm

In [3]:
# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("satviksh09/PnHLayman")
model = AutoModelForSeq2SeqLM.from_pretrained("satviksh09/PnHLayman")  

In [ ]:

def chunk_text(text, max_length=256):
    """Splits the input text into chunks of a specified maximum length."""
    words = text.split()
    chunks = []
    chunk = []
    
    for word in words:
        chunk.append(word)
        if len(chunk) > max_length:
            chunks.append(' '.join(chunk[:-1]))
            chunk = [chunk[-1]]
    if chunk:
        chunks.append(' '.join(chunk))
    return chunks


In [ ]:

def generate_summary(model, tokenizer, case_text, device="cuda:0"):
    # Tokenize the input text
    chunks = chunk_text(case_text, max_length=1024)

    all_summaries = []
    
    # Generate summary for each chunk
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding="longest", max_length=1024)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        summary_ids = model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            max_new_tokens=128,  # Adjust this as needed
            num_beams=8, 
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
            length_penalty=0.8
        )
        
        # Decode the summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        all_summaries.append(summary)
    
    # Combine the summaries from each chunk
    final_summary = " ".join(all_summaries)
    return final_summary

In [ ]:
# Insert judgment here.

case_text = """


"""

# Load text from a file instead of hardcoding it

with open("judgment.txt", "r", encoding="utf-8") as f:
    case_text = f.read()


In [ ]:
summary = generate_summary(model,tokenizer,case_text,device)
print(summary)